In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Step 1: Defining the Matching Framework
# Generate synthetic data for patients with repeated symptom measurements
data = pd.DataFrame({
    'id': range(1, 21),
    'urgency_baseline': np.random.randint(0, 10, 20),
    'urgency_t': np.random.randint(0, 10, 20),
    'frequency_baseline': np.random.randint(0, 10, 20),
    'frequency_t': np.random.randint(0, 10, 20),
    'pain_baseline': np.random.randint(0, 10, 20),
    'pain_t': np.random.randint(0, 10, 20),
    'age': np.random.randint(20, 80, 20),
    'gender': np.random.choice([0, 1], 20),  # 0 = Female, 1 = Male
    'treated': np.random.choice([0, 1], 20)  # 1 = treated, 0 = control
})

# Step 2: Risk Set Matching
# Separate treated and not-yet-treated patients
treated = data[data['treated'] == 1].reset_index(drop=True)
control = data[data['treated'] == 0].reset_index(drop=True)

# Compute covariance matrix and inverse for Mahalanobis distance
cov_matrix = np.cov(data[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']].T)
cov_inv = inv(cov_matrix)

def compute_distance_matrix(treated, control):
    """Computes Mahalanobis distance matrix between treated and control groups."""
    distances = {}
    for i, t in treated.iterrows():
        for j, c in control.iterrows():
            dist = mahalanobis(t[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               c[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               cov_inv)
            distances[(i, j)] = dist
    return distances

# Compute distance matrix
distance_matrix = compute_distance_matrix(treated, control)

# Step 3: Optimal Matching via Integer Programming
prob = LpProblem("Balanced_Matching", LpMinimize)

# Define decision variables
x = {pair: LpVariable(f"x_{pair[0]}_{pair[1]}", cat='Binary') for pair in distance_matrix.keys()}

# Objective function: Minimize total distance
prob += lpSum(distance_matrix[pair] * x[pair] for pair in distance_matrix.keys())

# Constraints: Each treated patient is matched to one control
for i in range(len(treated)):
    prob += lpSum(x[(i, j)] for j in range(len(control)) if (i, j) in x) == 1

# Each control patient is matched at most once
for j in range(len(control)):
    prob += lpSum(x[(i, j)] for i in range(len(treated)) if (i, j) in x) <= 1

# Step 4: Balancing Covariates
# Enforce balance constraints on binary variables (age, gender)
for attr in ['age', 'gender']:
    treated_sum = lpSum(treated.loc[i, attr] * x[(i, j)] for i, j in x.keys())
    control_sum = lpSum(control.loc[j, attr] * x[(i, j)] for i, j in x.keys())
    prob += treated_sum == control_sum

# Step 5: Ensuring Stability and Sensitivity Analysis
# Solve the optimization problem
prob.solve()

# Extract matched pairs
matches = [(i, j) for (i, j) in x.keys() if x[(i, j)].value() == 1]
print("Matched Pairs:", matches)


Optimal Pair Matching: [(np.int64(0), np.int64(381)), (np.int64(2), np.int64(247)), (np.int64(3), np.int64(340)), (np.int64(5), np.int64(387)), (np.int64(7), np.int64(70)), (np.int64(8), np.int64(186)), (np.int64(9), np.int64(229)), (np.int64(10), np.int64(113)), (np.int64(13), np.int64(48)), (np.int64(14), np.int64(61)), (np.int64(15), np.int64(370)), (np.int64(17), np.int64(112)), (np.int64(18), np.int64(151)), (np.int64(19), np.int64(191)), (np.int64(20), np.int64(133)), (np.int64(21), np.int64(29)), (np.int64(22), np.int64(396)), (np.int64(23), np.int64(35)), (np.int64(24), np.int64(341)), (np.int64(25), np.int64(241)), (np.int64(27), np.int64(309)), (np.int64(30), np.int64(236)), (np.int64(31), np.int64(181)), (np.int64(32), np.int64(53)), (np.int64(33), np.int64(364)), (np.int64(36), np.int64(58)), (np.int64(38), np.int64(26)), (np.int64(42), np.int64(374)), (np.int64(44), np.int64(279)), (np.int64(45), np.int64(274)), (np.int64(52), np.int64(398)), (np.int64(54), np.int64(335)),